# Übungssession zu Pandas

Erinnerung: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
LEGO = pd.read_csv("LEGO.csv")
LEGO.head(3)

### Frage 1: Preise
Ziel: Wie oft kommen (bei den aktuellen Modellen) die unterschiedlichen Preise jeweils vor? Sortieren Sie die Preise nach ihrer Häufigkeit.

In [ ]:
(LEGO[LEGO["date"] == "2022-09-19"]
    .groupby("price")
    .size()
    .sort_values(ascending = False))

### Einschub

Bisher hatten wir Split-Apply-Combine in einer relativ einfachen Form gesehen.
Meist wurde nach der Gruppierung eine Spalte ausgewählt und darauf eine Aggregationsfunktion angewandt.
Grundsätzlich kann man auch beliebige Funktionen auf die jeweiligen SubDataFrames anwenden.

In [ ]:
def some_stats(subdf):
    rows = subdf.shape[0]
    num_ages = subdf["ages"].nunique()
    num_prices = subdf["price"].nunique()
    total_price = subdf["price"].sum()
    return pd.Series([rows, num_ages, num_prices, total_price], index= ["num_sets", "num_ages", "num_prices", "total_price"])

LEGO[LEGO["date"]=="2022-09-19"].groupby("theme").apply(some_stats).sort_values("total_price", ascending = False)

### Frage 2: Teure Modelle

Ziel: Die Gesamttabelle (nur die aktuellen Modelle) soll eingeschränkt werden, so dass für jedes Thema nur die drei teuersten Modelle enthalten sind.

In [ ]:
(LEGO[LEGO["date"] == "2022-09-19"]
    .groupby("theme")
    .apply(lambda subdf: subdf.sort_values("price", ascending = False).iloc[0:3])
    .reset_index(drop = True))

### Einschub

Mit filter() kann man ganze Gruppen entfernen, sofern sie ein Kriterium nicht erfüllen.

In [ ]:
LEGO.groupby("theme").filter(lambda subdf: len(subdf) <= 2)

Mit transform() kann man eine Funktion aufrufen aber die Form der ursprünglichen Spalte belassen.
Dies ist meist hilfreich wenn man einen aggregierten Wert über alle Zeilen verteilen möchte, von denen er stammt.

In [ ]:
LEGO["meanprice_in_theme"] = LEGO.groupby("theme")["price"].transform("mean")
LEGO.head()

### Frage 3: Neue Modelle

Ziel: Welche Modelle sind im aktuellen Halbjahr neu hinzugekommen?

In [ ]:
df = LEGO.groupby("product_code").filter(lambda subdf: (len(subdf) == 1) & np.any(subdf["date"] == "2022-09-19"))
df.head()

In [ ]:
LEGO["occurences"] = LEGO.groupby("product_code")["date"].transform(len)
LEGO[(LEGO["occurences"] == 1) & (LEGO["date"] == "2022-09-19")].head()


### Frage 4: Schwierigkeitsgrad

(Wir betrachten nur die aktuellen Sets.)
Die Altersklasse gibt bereits eine gewisse Auskunft über den Schwierigkeitsgrad eines Sets. Aber wie verhalten sich diese untereinander?
Für jedes Modell soll ein Score ermittelt werden, der angibt wie schwierig es im Vergleich zu anderen Modellen der jeweiligen Altersklasse ist.

In [ ]:
LEGO_current = LEGO[LEGO["date"] == "2022-09-19"].copy()
LEGO_current["mean_pieces"] = LEGO_current.groupby("ages")["piece_count"].transform("median")
LEGO_current["rel_complexity"] = LEGO_current["piece_count"] / LEGO_current["mean_pieces"]
LEGO_current.head()